# Generate RADM_test sample


In [1]:
from src.model.minigpt4 import MiniGPT4
from src.processor.blip_processors import Blip2ImageTrainProcessor,Blip2ImageEvalProcessor,DinoImageProcessor
import torch
import json
import os
from PIL import Image
from html_to_ui import get_bbox
from generation import *
#with open("log_dir/train_stage2_with_augment_dino_codellama/generated_sample/16/test_numerical.jsonl", "r") as f:
#    content = [json.loads(line) for line in f]
#with open("data/cgl_dataset/for_posternuwa/html_format_img_instruct_all_mask_and_all_condition/test_numerical.jsonl", "r") as f:
#    sample = [json.loads(line) for line in f]
device = f"cuda:{1}" if torch.cuda.is_available() else "cpu"
vit_model_name = "dino_v2"
llama_model = "models/codeLlama-7b-hf"
base_model = "log_dir/train_stage2_with_augment_dino_codellama_text/checkpoints/checkpoint-16/pytorch_model.bin"
model = MiniGPT4(lora_r=64,low_resource=False,vit_model = vit_model_name,llama_model = llama_model)
model.load_state_dict(torch.load(base_model,map_location="cpu"))
model = model.to(device)
model.device = device
model.half()
model.eval()

image_processor = DinoImageProcessor()

In [2]:
from helper.global_var import *
def build_input(vals=None,text=None,canvas_width=513, canvas_height=750,task="unconditional") :
    """
        text = list of string
        vals = list of layout (category, xl,yl,xr,yr)
    """
    html_template = TEMPLATE_FORMAT.get("html_format")
    bbox_template = TEMPLATE_FORMAT.get("bbox_format")
    #instruction
    if text:
        task_instruction = TASK_INSTRUCTION.get("cgl")
        instruction = TEXT_INSTRUCTION.get(task)
        instruction = task_instruction + instruction
        
        t = []
        for te in text:
            t.append(te)
        text = " & ".join(text)
        
    else:
        task_instruction = TASK_INSTRUCTION.get("cgl")
        instruction = INSTRUCTION.get(task)
        instruction = task_instruction + instruction
    MASK= "<M>"
    def _build_rect(category='background', x=None, y=None, w=None, h=None):
        
        if not x:
            x="<M>"
        if not y:
            y="<M>"
        if not w:
            w="<M>"
        if not h:
            h = "<M>"
        
        rect = bbox_template.format(c=category, x=x, y=y, w=w, h=h)
        return rect #f'<rect data-category=\"{category}\", x=\"{x}\", y=\"{y}\", width=\"{w}\", height=\"{h}\"/>\n'
    
    
    contents = []
    if vals:
        for category, x, y, w, h in vals :
            contents.append(_build_rect(category, x, y, w, h))
            content = "\n".join(contents)
    else:
        content = ""
    bbox_html = html_template.format(W=canvas_width, H=canvas_height, content = content)
    #str_output = f'{instruction} (in html format):\n###bbox html:  <body> <svg width=\"{canvas_width}\" height=\"{canvas_height}\">{rects} </svg> </body>'
    if text:
        str_output = instruction.format(text= text, bbox_html = bbox_html)
    else:
        str_output = instruction.format(bbox_html = bbox_html)
    return str_output
text = None#["ECCV 2024","PosterLlama"]
input = build_input(None,None)
input

'I want to generate layout in poster design format. plaese generate the layout html according to the image I provide (in html format):\n###bbox html: <body> <svg width="513" height="750">  </svg> </body>'

In [2]:
import json
with open("/data1/poong/tjfwownd/PosterNUWA/data/cgl_dataset/radm_dataset/RADM_dataset/text_context/test_translated_text.json","r") as f:
    test_translated = json.load(f)
translated_dict = {}
for test in test_translated:
    key = list(test.keys())[0]
    value = list(test.values())[0]
    translated_dict[key]=value
    

build_input(None,translated_dict[list(translated_dict.keys())[0]])

'I want to generate layout in poster design format. plaese generate the layout html according to the image I provide (in html format)\nText: Built-in WIFI & Pantum Printer & Unique rounded design & Top styling & Laser printing without clogging & Efficient, no double images\n###bbox html: <body> <svg width="513" height="750">  </svg> </body>'

In [3]:
import matplotlib.pyplot as plt
import os
from PIL import Image, ImageFilter
DATASET_COLOR = {
    1: '#929F29',   
    2: '#1FA39A',  
    3: '#987FF2',      
    4: '#F56881',    
    5: "#0000FF"      
}
def sort_key(x):
    if x[0] == 3:
        return -1
    else:
        return x[0]
img_paths = list(translated_dict.keys())  #os.listdir("/data1/poong/tjfwownd/PosterNUWA/data/cgl_dataset/radm_dataset/RADM_dataset/images/test")
si = 0
output_file = f"/data1/poong/tjfwownd/PosterNUWA/log_dir/Paper_samples/Generated_Sample_RADM_test/PosterLlama/{si}"

os.makedirs(os.path.join(output_file,"generated_sample"),exist_ok=True)


#uncond_inst = 'I want to generate layout in poster design format.plaese generate the layout html according to the image I provide (in html format):\n###bbox html: <body> <svg width="513" height="750">  </svg> </body> '
#inst = [uncond_inst]*len(img_paths)

boxx = []
clx = []
imgs = []
save_img=True
from tqdm import tqdm
import random

used_json=[]

im_dir = "/data1/poong/tjfwownd/PosterNUWA/data/cgl_dataset/radm_dataset/RADM_dataset/images/test"

for path in tqdm(img_paths):
    img = Image.open(os.path.join(im_dir,path)).resize((513,750))
    processed_images = image_processor(img).unsqueeze(0)

    texts = translated_dict[path]
    N = random.randint(1,len(texts))
    texts = random.sample(texts,N)
    inst = [build_input(None,texts)]
    
    with torch.no_grad():
        with torch.autocast(device_type="cuda"):
            samp = model.generate(processed_images, inst,max_new_tokens=512,do_sample=False,temperature=0.6,top_p=0.9)
    
    bboxes,clses = get_bbox(samp[0])
    used_json.append({path : texts})
    if save_img:
        drawbbox = []
        for bbox in bboxes:
            x1, y1, x2, y2 = bbox
            x2 += x1
            y2 += y1
            bbox  = [x1, y1, x2, y2]
            drawbbox.append(np.array(bbox))
        drawn_outline = img.copy()
        drawn_fill = img.copy()
        draw_ol = ImageDraw.ImageDraw(drawn_outline)
        draw_f = ImageDraw.ImageDraw(drawn_fill)

        sorted_pairs = sorted(zip(clses, drawbbox), key=sort_key)
        sorted_classes, sorted_bboxes = zip(*sorted_pairs)

        clses = list(sorted_classes)
        drawbbox = list(sorted_bboxes)

        for b,l in zip(drawbbox,clses):
            draw_ol.rectangle([b[0],b[1],b[2],b[3]], outline=DATASET_COLOR[l], width=2)
        for b,l in zip(drawbbox,clses):
            draw_f.rectangle([b[0],b[1],b[2],b[3]], fill=DATASET_COLOR[l])
        drawn_outline = drawn_outline.convert("RGBA")
        drawn_fill = drawn_fill.convert("RGBA")

        drawn_fill.putalpha(int(256 * 0.4))
        img = Image.alpha_composite(drawn_outline, drawn_fill)
        #layout_images.append(img)
        #display(img)
        img.convert("RGB").save(os.path.join(output_file,"generated_sample",path))

    boxx.append(np.array(drawbbox).tolist())
    clx.append(np.array(clses).tolist())
    imgs.append(path)

if  save_img:
    display(img)
    
with open(os.path.join(output_file,"box.json"), "w") as f:
    json.dump(boxx,f,indent=2)

with open(os.path.join(output_file,"clses.json"),"w") as f:
    json.dump(clx,f,indent=2)

with open(os.path.join(output_file,"text_order.json"),"w") as f:
    json.dump(imgs,f,indent=2)

with open(os.path.join(output_file,"used_text.json"),"w") as f:
    json.dump(used_json,f,indent=2)

  0%|          | 0/995 [00:00<?, ?it/s]


NameError: name 'Image' is not defined

In [144]:
import random
texts = translated_dict[img_paths[660]] #translated_dict[path]
N = random.randint(1,len(texts))
texts = random.sample(texts,N)
inst = [build_input(None,texts)]
inst,N,len(texts)
    

(['I want to generate layout in poster design format. plaese generate the layout html according to the image I provide (in html format)\nText: Crunchy, Rich Wheat Aroma & Relentlessly Crunchy, Delightful Taste & Irresistibly Crisp, Can\'t Stop Munching & \n###bbox html: <body> <svg width="513" height="750">  </svg> </body>'],
 4,
 4)

In [151]:
texts = ['Crunchy, Rich Wheat Aroma',
 'Relentlessly Crunchy, Delightful Taste',
 "Irresistibly Crisp, Can't Stop Munching",
 '','']
texts

['Crunchy, Rich Wheat Aroma',
 'Relentlessly Crunchy, Delightful Taste',
 "Irresistibly Crisp, Can't Stop Munching",
 '',
 '']

In [152]:
texts = [item for item in texts if item != '']
texts

['Crunchy, Rich Wheat Aroma',
 'Relentlessly Crunchy, Delightful Taste',
 "Irresistibly Crisp, Can't Stop Munching"]

# Make augment jsonpath

In [16]:
import json
with open("/data1/poong/tjfwownd/PosterNUWA/data/cgl_dataset/radm_dataset/RADM_dataset/text_context/test_translated_text.json","r") as f:
    test_translated = json.load(f)

aug_test_translated = []

for test in test_translated:
    print(list(test.keys())[0],list(test.values())[0])
    print(list(test.keys())[0].replace(".png","_aug0.png"))
    aug_test_translated.append({list(test.keys())[0].replace(".png","_aug0.png") : list(test.values())[0]})
    aug_test_translated.append({list(test.keys())[0].replace(".png","_aug1.png") : list(test.values())[0]})
    aug_test_translated.append({list(test.keys())[0].replace(".png","_aug2.png") : list(test.values())[0]})

with open("/data1/poong/tjfwownd/PosterNUWA/data/cgl_dataset/radm_dataset/RADM_augmentation_test/test_translated_text.json","w") as f:
    json.dump(aug_test_translated,f,indent=1)

100022019440_10_mask002.png ['Built-in WIFI', 'Pantum Printer', 'Unique rounded design', 'Top styling', 'Laser printing without clogging', 'Efficient, no double images']
100022019440_10_mask002_aug0.png
100022019440_1_mask002.png ['Built-in WIFI', 'Pantum Printer', 'Unique rounded design', 'Top cover styling', 'Laser printing, no clog', 'Efficient, no double image']
100022019440_1_mask002_aug0.png
100013499087_9_mask002.png ['Savor the Moment with Spicy Strips', 'Convenient Packs for Hygienic Snacking']
100013499087_9_mask002_aug0.png
100014992589_1_mask002.png ['Dual-Color Injection Molding', 'Power-Packed Long-lasting Battery', 'Magnetic Palm Rest for Enduring Comfort', 'Wireless Light Sync', 'Ultron Software']
100014992589_1_mask002_aug0.png
100020344345_2_mask002.png ['High-Performance Recommended', 'Lenovo Workstation', 'Honeycomb Air Intake Design', 'Triple Channel Cooling Technology', 'Dual-Core Flagship']
100020344345_2_mask002_aug0.png
100016198564_14_mask002.png ['Removable f

In [10]:
import json
import copy
with open("/data1/poong/tjfwownd/PosterNUWA/data/cgl_dataset/radm_dataset/RADM_dataset/annotations/test.json", "r") as f:
    sample = json.load(f)

aug_sample = {"images" : [],
              "annotations" : [],
              "categories" : sample['categories']}
id = 0
for im in sample['images']:
    aug1 = copy.deepcopy(im)
    aug2 = copy.deepcopy(im)
    aug3 = copy.deepcopy(im)
    aug1['url'] = im['url'].replace(".png","_aug0.png")
    aug1['file_name'] = im['file_name'].replace(".png","_aug0.png")
    aug1['id'] = id
    aug_sample['images'].append(aug1)
    id+=1
    aug2['url'] = im['url'].replace(".png","_aug1.png")
    aug2['file_name'] = im['file_name'].replace(".png","_aug1.png")
    aug2['id'] = id
    aug_sample['images'].append(aug2)
    id+=1
    aug3['url'] = im['url'].replace(".png","_aug2.png")
    aug3['file_name'] = im['file_name'].replace(".png","_aug2.png")
    aug3['id'] = id
    aug_sample['images'].append(aug3)
    id+=1
    
with open("/data1/poong/tjfwownd/PosterNUWA/data/cgl_dataset/radm_dataset/RADM_augmentation_test/annotations/test.json","w") as f:
    json.dump(aug_sample,f,indent=1)

In [13]:
aug_sample

{'images': [{'url': '100000002065_16_mask002_aug0.png',
   'file_name': '100000002065_16_mask002_aug0.png',
   'id': 0,
   'width': 350,
   'height': 520},
  {'url': '100000002065_16_mask002_aug1.png',
   'file_name': '100000002065_16_mask002_aug1.png',
   'id': 1,
   'width': 350,
   'height': 520},
  {'url': '100000002065_16_mask002_aug2.png',
   'file_name': '100000002065_16_mask002_aug2.png',
   'id': 2,
   'width': 350,
   'height': 520},
  {'url': '100000022707_1_mask002_aug0.png',
   'file_name': '100000022707_1_mask002_aug0.png',
   'id': 3,
   'width': 350,
   'height': 520},
  {'url': '100000022707_1_mask002_aug1.png',
   'file_name': '100000022707_1_mask002_aug1.png',
   'id': 4,
   'width': 350,
   'height': 520},
  {'url': '100000022707_1_mask002_aug2.png',
   'file_name': '100000022707_1_mask002_aug2.png',
   'id': 5,
   'width': 350,
   'height': 520},
  {'url': '100000196401_2_mask002_aug0.png',
   'file_name': '100000196401_2_mask002_aug0.png',
   'id': 6,
   'width': 